# Dynamo-based perturbation analysis

Notebooks for predicting TF perturbation effects using dynamo.

## Library imports

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import mplscience
import seaborn as sns

import dynamo as dyn
import scanpy as sc
from dynamo.preprocessing import Preprocessor

from rgv_tools import DATA_DIR, FIG_DIR
from rgv_tools.perturbation import (
    abundance_test,
    get_list_name,
    Multiple_TFScanning_KO_dyn,
    Multiple_TFScanning_perturbation_dyn,
    split_elements,
    TFScanning_KO_dyn,
    TFScanning_perturbation_dyn,
)

/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `anndata` is deprecated. Import anndata.io.read_mtx instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packag

## Constants

In [2]:
DATASET = "zebrafish"

In [3]:
SAVE_DATA = True
if SAVE_DATA:
    (DATA_DIR / DATASET / "results").mkdir(parents=True, exist_ok=True)

In [4]:
SAVE_FIGURES = False
if SAVE_FIGURES:
    (FIG_DIR / DATASET).mkdir(parents=True, exist_ok=True)

In [5]:
genes = ["nr2f5", "sox9b", "twist1b", "ets1"]

TERMINAL_STATES = [
    "mNC_head_mesenchymal",
    "mNC_arch2",
    "mNC_hox34",
    "Pigment",
]

In [6]:
single_ko = ["elk3", "erf", "fli1a", "mitfa", "nr2f5", "rarga", "rxraa", "smarcc1a", "tfec", "nr2f2"]
multiple_ko = ["fli1a_elk3", "mitfa_tfec", "tfec_mitfa_bhlhe40", "fli1a_erf_erfl3", "erf_erfl3"]

## Data loading

In [7]:
adata = sc.read_h5ad(DATA_DIR / DATASET / "processed" / "adata_preprocessed.h5ad")

In [8]:
adata.X = adata.layers["matrix"].copy()

## Processing by dynamo

In [9]:
preprocessor = Preprocessor()
preprocessor.preprocess_adata(adata, recipe="monocle")

|-----> Running monocle preprocessing pipeline...
|-----------> filtered out 0 outlier cells
|-----------> filtered out 21 outlier genes
|-----> PCA dimension reduction
|-----> <insert> X_pca to obsm in AnnData Object.
|-----> [Preprocessor-monocle] completed [1.1507s]


In [10]:
dyn.tl.dynamics(adata)

|-----> dynamics_del_2nd_moments_key is None. Using default value from DynamoAdataConfig: dynamics_del_2nd_moments_key=False
|-----------> removing existing M layers:[]...
|-----------> making adata smooth...
|-----> calculating first/second moments...
|-----? layer X_velocity is not in any of the (['X_spliced', 'X_unspliced'], ['X_new', 'X_total'], ['X_uu', 'X_ul', 'X_su', 'X_sl']) groups, skipping...
|-----? layer X_variance_velocity is not in any of the (['X_spliced', 'X_unspliced'], ['X_new', 'X_total'], ['X_uu', 'X_ul', 'X_su', 'X_sl']) groups, skipping...
|-----? layer X_Mu is not in any of the (['X_spliced', 'X_unspliced'], ['X_new', 'X_total'], ['X_uu', 'X_ul', 'X_su', 'X_sl']) groups, skipping...
|-----? layer X_Ms is not in any of the (['X_spliced', 'X_unspliced'], ['X_new', 'X_total'], ['X_uu', 'X_ul', 'X_su', 'X_sl']) groups, skipping...
|-----? layer X_Mu is not in any of the (['X_spliced', 'X_unspliced'], ['X_new', 'X_total'], ['X_uu', 'X_ul', 'X_su', 'X_sl']) groups, ski

estimating gamma: 100%|██████████| 988/988 [00:10<00:00, 96.84it/s] 


AnnData object with n_obs × n_vars = 697 × 988
    obs: 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'cell_type', 'stage', 'cell_type2', 'macrostates', 'nGenes', 'nCounts', 'pMito', 'pass_basic_filter', 'unspliced_Size_Factor', 'initial_unspliced_cell_size', 'Size_Factor', 'initial_cell_size', 'spliced_Size_Factor', 'initial_spliced_cell_size', 'variance_velocity_Size_Factor', 'initial_variance_velocity_cell_size', 'Ms_Size_Factor', 'initial_Ms_cell_size', 'velocity_Size_Factor', 'initial_velocity_cell_size', 'Mu_Size_Factor', 'initial_Mu_cell_size', 'ntr'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'gene_count_corr', 'is_tf', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes', 'TF', 'nCells', 'nCounts', 'pass_basic_filter', 'frac', 'use_for_pca', 'ntr', 'beta', 'gamma', 'half_life', 'alpha_b', 'alpha_r2', 'gamma_b', 'gamma_r2', 'gamma_logLL', 'delta_b', 'delta_r2', 'bs', 'bf', 'uu0', 'ul0', 'su0', 'sl0', 'U0', 'S

In [11]:
dyn.tl.reduceDimension(adata)

|-----> retrieve data for non-linear dimension reduction...
|-----? adata already have basis umap. dimension reduction umap will be skipped! 
set enforce=True to re-performing dimension reduction.
|-----> [UMAP] completed [0.0022s]


In [12]:
dyn.tl.cell_velocities(adata, basis="pca")

|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [0.5832s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.3028s]


AnnData object with n_obs × n_vars = 697 × 988
    obs: 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'cell_type', 'stage', 'cell_type2', 'macrostates', 'nGenes', 'nCounts', 'pMito', 'pass_basic_filter', 'unspliced_Size_Factor', 'initial_unspliced_cell_size', 'Size_Factor', 'initial_cell_size', 'spliced_Size_Factor', 'initial_spliced_cell_size', 'variance_velocity_Size_Factor', 'initial_variance_velocity_cell_size', 'Ms_Size_Factor', 'initial_Ms_cell_size', 'velocity_Size_Factor', 'initial_velocity_cell_size', 'Mu_Size_Factor', 'initial_Mu_cell_size', 'ntr'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'gene_count_corr', 'is_tf', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes', 'TF', 'nCells', 'nCounts', 'pass_basic_filter', 'frac', 'use_for_pca', 'ntr', 'beta', 'gamma', 'half_life', 'alpha_b', 'alpha_r2', 'gamma_b', 'gamma_r2', 'gamma_logLL', 'delta_b', 'delta_r2', 'bs', 'bf', 'uu0', 'ul0', 'su0', 'sl0', 'U0', 'S

In [13]:
dyn.vf.VectorField(adata, basis="pca")

|-----> VectorField reconstruction begins...
|-----> Retrieve X and V based on basis: PCA. 
        Vector field will be learned in the PCA space.
|-----> Learning vector field with method: sparsevfc.
|-----> [SparseVFC] begins...
|-----> Sampling control points based on data velocity magnitude...
|-----> [SparseVFC] completed [0.1569s]
|-----> [VectorField] completed [0.2136s]


In [14]:
adata_perturb = adata.copy()

In [15]:
del adata.uns["cell_type_colors"]

## Compare predict fate probability changes

In [16]:
fate_prob = {}
fate_prob_perturb = {}

for g in genes:
    fb, fb_perturb = TFScanning_KO_dyn(adata, 8, "cell_type", TERMINAL_STATES, [g], fate_prob_return=True)
    fate_prob[g] = fb
    fate_prob_perturb[g] = fb_perturb

  0%|          | 0/697 [00:00<?, ?cell/s]

  0%|          | 0/697 [00:00<?, ?cell/s]

2024-11-24 16:02:13,722 - INFO - Using pre-computed Schur decomposition


Defaulting to `'gmres'` solver.


  0%|          | 0/4 [00:00<?, ?/s]

...

  0%|          | 0/697 [00:00<?, ?cell/s]

  0%|          | 0/697 [00:00<?, ?cell/s]

2024-11-24 16:02:38,796 - INFO - Using pre-computed Schur decomposition


  0%|          | 0/4 [00:00<?, ?/s]

Done ets1


## Visualize dynamo perturbation effects

In [17]:
for g in genes:
    data = abundance_test(fate_prob[g], fate_prob_perturb[g])
    data = pd.DataFrame(
        {
            "Score": data.iloc[:, 0].tolist(),
            "p-value": data.iloc[:, 1].tolist(),
            "Terminal state": data.index.tolist(),
            "TF": [g] * (data.shape[0]),
        }
    )

    final_df = data.copy()
    final_df["Score"] = 0.5 - final_df["Score"]

    color_label = "cell_type"
    df = pd.DataFrame(final_df["Score"])
    df.columns = ["coefficient"]
    df["Cell type"] = final_df["Terminal state"]
    order = df["Cell type"].tolist()

    palette = dict(zip(adata.obs[color_label].cat.categories, adata_perturb.uns[f"{color_label}_colors"]))
    subset_palette = {name: color for name, color in palette.items() if name in TERMINAL_STATES}

    with mplscience.style_context():
        sns.set_style(style="whitegrid")
        fig, ax = plt.subplots(figsize=(2, 2))
        sns.barplot(
            data=df,
            y="coefficient",
            x="Cell type",
            palette=subset_palette,
            order=order,
            ax=ax,
        )
        ax.tick_params(axis="x", rotation=90)
        plt.title("$\\mathit{" + g + "}$ regulon knock out simulation")

        if SAVE_FIGURES:
            plt.savefig(
                FIG_DIR / DATASET / f"{g}_perturbation_simulation_dynamo.svg",
                format="svg",
                transparent=True,
                bbox_inches="tight",
            )
        # Show the plot
        plt.show()

## Perturbation prediction

### Single gene knockout

In [18]:
single_ko = set(single_ko).intersection(adata.var_names)
single_ko = list(single_ko)

In [19]:
## Dynamo (KO)
ko_dyn = TFScanning_KO_dyn(adata, 8, "cell_type", TERMINAL_STATES, single_ko)

## Dynamo (perturbation)
perturbation_dyn = TFScanning_perturbation_dyn(adata, 8, "cell_type", TERMINAL_STATES, single_ko)

  0%|          | 0/697 [00:00<?, ?cell/s]

  0%|          | 0/697 [00:00<?, ?cell/s]

2024-11-24 16:02:44,048 - INFO - Using pre-computed Schur decomposition


  0%|          | 0/4 [00:00<?, ?/s]

|-----> In silico knockout nr2f5
|-----> Project the high dimensional vector field after KO to umap.
|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [0.2852s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.2654s]


...

  0%|          | 0/4 [00:00<?, ?/s]

Done tfec
|-----> In silico perturbation of single-cells and prediction of cell fate after perturbation...
|-----> Retrive X_pca, PCs, pca_mean...
|-----> Calculate perturbation effect matrix via \delta Y = J \dot \delta X....


calculating Jacobian for each cell: 100%|██████████| 697/697 [00:00<00:00, 109742.48it/s]

|-----> project the pca perturbation vector to low dimensional space....


|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.3244s]
|-----> you can use dyn.pl.streamline_plot(adata, basis='pca_perturbation') to visualize the perturbation vector
Done nr2f5
|-----> In silico perturbation of single-cells and prediction of cell fate after perturbation...
|-----> Retrive X_pca, PCs, pca_mean...
|-----> Calculate perturbation effect matrix via \delta Y = J \dot \delta X....
|-----> project the pca perturbation vector to low dimensional space....
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.2364s]
|-----> you can use dyn.pl.streamline_plot(adata, basis='pca_perturbation') to visualize the perturbation vector
Done nr2f2
|-----> In silico perturbation of single-cells and prediction of cell fate after perturbation...
|-----> Retrive X

In [20]:
## Perform KO screening using function based perturbation
coef_KO = pd.DataFrame(np.array(ko_dyn["coefficient"]))
coef_KO.index = ko_dyn["TF"]
coef_KO.columns = get_list_name(ko_dyn["coefficient"][0])

## Perform perturbation screening using gene expression based perturbation
coef_perturb = pd.DataFrame(np.array(perturbation_dyn["coefficient"]))
coef_perturb.index = perturbation_dyn["TF"]
coef_perturb.columns = get_list_name(perturbation_dyn["coefficient"][0])

### Multiple gene knock-out prediction

In [21]:
multiple_ko_list = split_elements(multiple_ko)

In [22]:
## Dynamo (KO)
ko_dyn = Multiple_TFScanning_KO_dyn(adata, 9, "cell_type", TERMINAL_STATES, multiple_ko_list)

## Dynamo (perturbation)
perturbation_ko = Multiple_TFScanning_perturbation_dyn(adata, 9, "cell_type", TERMINAL_STATES, multiple_ko_list)

  0%|          | 0/697 [00:00<?, ?cell/s]

  0%|          | 0/697 [00:00<?, ?cell/s]

2024-11-24 16:03:33,803 - INFO - Using pre-computed Schur decomposition


  0%|          | 0/4 [00:00<?, ?/s]

|-----> In silico knockout ['fli1a', 'elk3']
|-----> Project the high dimensional vector field after KO to umap.
|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [0.3093s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.2854s]


...

  0%|          | 0/697 [00:00<?, ?cell/s]

  0%|          | 0/697 [00:00<?, ?cell/s]

2024-11-24 16:03:57,243 - INFO - Using pre-computed Schur decomposition


  0%|          | 0/4 [00:00<?, ?/s]

['erf', 'erfl3']
Done erf_erfl3
|-----> In silico perturbation of single-cells and prediction of cell fate after perturbation...
|-----> Retrive X_pca, PCs, pca_mean...
|-----> Calculate perturbation effect matrix via \delta Y = J \dot \delta X....
|-----> project the pca perturbation vector to low dimensional space....
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.2709s]
|-----> you can use dyn.pl.streamline_plot(adata, basis='umap_perturbation') to visualize the perturbation vector
Done fli1a_elk3
|-----> In silico perturbation of single-cells and prediction of cell fate after perturbation...
|-----> Retrive X_pca, PCs, pca_mean...
|-----> Calculate perturbation effect matrix via \delta Y = J \dot \delta X....
|-----> project the pca perturbation vector to low dimensional space....
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0

In [23]:
## Perform KO screening using function based perturbation
coef_KO_multiple = pd.DataFrame(np.array(ko_dyn["coefficient"]))
coef_KO_multiple.index = ko_dyn["TF"]
coef_KO_multiple.columns = get_list_name(ko_dyn["coefficient"][0])

## Perform perturbation screening using gene expression based perturbation
coef_perturb_multiple = pd.DataFrame(np.array(perturbation_dyn["coefficient"]))
coef_perturb_multiple.index = perturbation_dyn["TF"]
coef_perturb_multiple.columns = get_list_name(perturbation_dyn["coefficient"][0])

## Save dataset

In [24]:
if SAVE_DATA:
    coef_KO.to_csv(DATA_DIR / DATASET / "results" / "dynamo_KO_single.csv")
    coef_KO_multiple.to_csv(DATA_DIR / DATASET / "results" / "dynamo_KO_multiple.csv")

    coef_perturb.to_csv(DATA_DIR / DATASET / "results" / "dynamo_perturb_single.csv")
    coef_perturb_multiple.to_csv(DATA_DIR / DATASET / "results" / "dynamo_perturb_multiple.csv")